In [1]:
from dgl.data.utils import save_graphs
import dgl
import torch as th
import scipy.sparse as sp
import networkx as nx
import pickle
from numpy import array
import pandas as pd

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

#DataFrame
import pandas as pd

#Rimozione duplicati lista
from collections import OrderedDict

#Aprire i File
import os
#Per la gestione dei file già esistenti
from os.path import exists

#Salvataggio dizinario
import pickle 

from sklearn import preprocessing

import csv

Si cambia il path per decidere quale dataset creare in funzione della GNN.

In [2]:
path = "../../R/Rdati/Matrici Adiacenza_otto_classi/"

In [3]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

Amino = []
ID_Graph = []
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            
            id_p = filename[0:4]
            ID_Graph.append(id_p)
            matrix = "../../R/Rdati/Matrici Adiacenza_otto_classi/{}.csv".format(id_p)            
            proteina = pd.read_csv(matrix, sep = ';')
            Amino.extend(list(proteina.iloc[0][1:]))


Amino_TOT = list(OrderedDict.fromkeys(Amino))

classes = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']

"""if not exists("../Dati/Dataset/aminoacidi.pickle"):
    aminoAcidi = preprocessing.LabelEncoder()
    aminoAcidi.fit(Amino_TOT)
    f = open("../Dati/Dataset/aminoacidi.pickle","wb")
    pickle.dump(aminoAcidi, f)
    f.close()

if not exists("../Dati/Dataset/classi.pickle"):
    classi = preprocessing.LabelEncoder()
    classi.fit(classes)
    f = open("../Dati/Dataset/classi.pickle","wb")
    pickle.dump(classi, f)
    f.close()

if not exists("../Dati/Dataset/ID_Graphs.pickle"):
    ID_Graphs = preprocessing.LabelEncoder()
    ID_Graphs.fit(ID_Graph)
    f = open("../Dati/Dataset/ID_Graphs.pickle","wb")
    pickle.dump(ID_Graphs, f)
    f.close()"""

aminoAcidi = preprocessing.LabelEncoder()
aminoAcidi.fit(Amino_TOT)
f = open("../Dati/Dataset_otto_classi/aminoacidi.pickle","wb")
pickle.dump(aminoAcidi, f)
f.close()

classi = preprocessing.LabelEncoder()
classi.fit(classes)
f = open("../Dati/Dataset_otto_classi/classi.pickle","wb")
pickle.dump(classi, f)
f.close()


ID_Graphs = preprocessing.LabelEncoder()
ID_Graphs.fit(ID_Graph)
f = open("../Dati/Dataset_otto_classi/ID_Graphs.pickle","wb")
pickle.dump(ID_Graphs, f)
f.close()

In [4]:
proteine_path = "../Dati/proteine.csv"
proteine = pd.read_csv(proteine_path, sep = ',')

f = open("../Dati/Dataset_otto_classi/classi.pickle","rb")
classi = pickle.load(f)
f.close()

f = open("../Dati/Dataset_otto_classi/aminoacidi.pickle","rb")
amino = pickle.load(f)
f.close()

f = open("../Dati/Dataset_otto_classi/ID_Graphs.pickle","rb")
ID_Graphs_read = pickle.load(f)
f.close()

ID_Graph = []

header_edges = ["graph_id", "src_id", "dst_id"]
f_e = open("../Dati/Dataset_otto_classi/edges.csv", 'w')
writer_e = csv.writer(f_e)
writer_e.writerow(header_edges)

header_nodes = ["graph_id", "node_id", "label"]
f_n = open("../Dati/Dataset_otto_classi/nodes.csv", 'w')
writer_n = csv.writer(f_n)
writer_n.writerow(header_nodes)

header_graphs = ["graph_id", "label"]
f_g = open("../Dati/Dataset_otto_classi/graphs.csv", 'w')
writer_g = csv.writer(f_g)
writer_g.writerow(header_graphs)

#edges = pd.DataFrame(columns=["graph_id", "src_id", "dst_id"])

#nodes = pd.DataFrame(columns=["graph_id", "node_id", "label"])

#graphs = pd.DataFrame(columns=["graph_id", "label"])


for filename in os.listdir("../../R/Rdati/Matrici Adiacenza_otto_classi/"):
    with open(os.path.join("../../R/Rdati/Matrici Adiacenza_otto_classi/", filename), 'r', encoding='windows-1252') as f:
        if (filename != ".DS_Store"):
            id_p = filename[0:4]
            #print(id_p)

            #Id della proteina che si sta analizzando
            print(id_p)

            if id_p not in ID_Graph: 
                ID_Graph.append(id_p)
                matrix = "../../R/Rdati/Matrici Adiacenza_otto_classi/{}.csv".format(id_p)
                #print(matrix)

                #Matrice adiacenza singola matrice inserita in un dataframe
                proteina = pd.read_csv(matrix, sep = ';')
                #Nomi dei nodi nonché numero
                Nodes = list(proteina.columns[1:])
                
                for vertex in range(0,len(Nodes)):   
                    Nodes[vertex] = Nodes[vertex][1:]
                    
                #Lista degli Amino acidi
                    #Amino = list(proteina.iloc[0][1:])
                Amino = amino.transform(list(proteina.iloc[0][1:]))

                
                proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                j = proteina.iloc[0]
                proteina.drop([0], axis = 0, inplace = True)
                
                
                    #proteina.drop(proteina.columns[[0]], axis = 1, inplace = True)
                    #proteina.drop([0], axis = 0, inplace = True)
                #Classe della proteina
                classe = proteine[proteine['Id'] == id_p]["Classe"].iloc[0][0]
        
                
                #Creazione Grafo
                G = nx.Graph()
                G.add_nodes_from(Nodes)
                
                #Aggiunta archi
                rows, cols = proteina.shape
                Vprimo = "Vx"
                Vsecondo ="Vx"
                for i in range(rows):
                    row = proteina.iloc[i]
                    j = 0
                    for edge in row:
                        if edge == '1':
                            vPrimo = Vprimo[:1] + str(i+1)
                            vSecondo = Vsecondo[:1] + str(j+1)
                            #print("Aggiunta arco tra il nodo ", vPrimo[1:],"e il nodo ", vSecondo[1:])
                            G.add_edge(vPrimo[1:], vSecondo[1:])
                        j += 1
                
                #Conbersione Grafo in formato DGL
                new_g = dgl.from_networkx(G)
                
                
                
                #Creazione file CSV Grafi
                id_graph = ID_Graphs_read.transform([id_p])[0]

                a = new_g.edges()[0]
                b = new_g.edges()[1]
                src_id = a.tolist()
                dst_id = b.tolist()
                    
                
                graph_id = []
                for i in range(0,len(src_id)):
                    graph_id.append(id_graph)
                
                """# Create dict edge
                data = {'graph_id': graph_id,
                        'src_id': src_id,
                        'dst_id': dst_id
                       }

                # Create DataFrame
                edge = pd.DataFrame(data)
                
                edges = edges.merge(edge, how='outer')
                """
                
                #Inserimento informazioni archi
                for i in range(0,len(graph_id)):
                    row = [graph_id[i], src_id[i], dst_id[i]]
                    writer_e.writerow(row)
                #Si potrebbe chiudere qui la connessione con f_e
                # f_e.close()
                
                
                node = new_g.nodes()
                node_id = node.tolist()
                graph_id = graph_id[:len(Amino)]

                
                Amino = Amino.astype("float")
                Amino = Amino.astype("str")
                
                """# Create dict node
                data = {'graph_id': graph_id,
                        'node_id': node_id,
                        'label': Amino
                       }
                # Create DataFrame
                node = pd.DataFrame(data)
                nodes = nodes.merge(node, how = 'outer')"""
                
                #Questa porzione di codice serve per introdurre la virgola nei file
                end = ",0.0" #si può levare lo 0 solo che potrebbero esserci conflitti
                for i in range(0,len(Amino)):
                    Amino[i] =Amino[i]+end
                #######
                
                
                #Inserimento informazioni nodi
                for i in range(0,len(graph_id)):
                    row = [graph_id[i], node_id[i], Amino[i]]
                    writer_n.writerow(row)
                #Si potrebbe chiudere qui la connessione con f_n
                # f_n.close()
                
        
                classe = classi.transform([classe])
                graph_id = graph_id[:1]

                
                classe = classe.astype("float")
                """ # Create dict graph
                data = {'graph_id': graph_id,
                        'label': classe
                       }

                # Create DataFrame
                graph = pd.DataFrame(data)
                
                graphs = graphs.merge(graph, how = 'outer')"""
            
                #Inserimento informazioni grafi
                for i in range(0,len(graph_id)):
                    row = [graph_id[i], classe[i]]
                    writer_g.writerow(row)
                #Si potrebbe chiudere qui la connessione con f_g 
                # f_g.close()
                
#edges.to_csv('../Dati/Dataset/edges.csv', index=False)
#nodes.to_csv('../Dati/Dataset/nodes.csv', index=False)
#graphs.to_csv('../Dati/Dataset/graphs.csv', index=False)


f_e.close()
f_n.close()
f_g.close()

3fb5
1bxl
3sy0
3agi
5ec2
5b1z
1g1z
1ikq
5ujw
4vhb
2roc
1n7s
4dte
6uvc
3ajn
1wmu
1iem
4odc
4hw2
1uck
4mkn
1ban
6b4l
1ird
1r3j
2kux
1bbb
1nwn
1r14
1rge
1kil
1b9b
1eit
6lcw
3tim
2ndb
1ri8
1u8k
1sar
1db2
1cdu
2hi5
4e3k
1ce0
1jl6
1t2h
1or4
1cdb
1cg8
2qfn
2a5y
1t0w
2hds
2vog
2zyp
1fvc
2oay
4lv2
1dxj
5vx1
4wba
6qb4
1vga
1dzb
1a2p
7npk
1jvn
2vxn
1mss
1dv7
4ifm
2oyh
5oxr
3ly4
2j27
1jzm
1hbi
4uuj
6stm
1q9k
1hbh
1jzl
1tqx
1aif
5oxs
3idm
1svz
1swi
2oyi
1fll
3iga
1o07
1q6r
4lve
1jtp
4lv3
1jtg
1mgw
1cid
2o7h
6cj7
1dly
2hdr
2vof
2mxo
1dn0
1ap2
1ash
1s61
6t3d
3kj2
5zv6
2v5b
4rds
7wga
5jr5
1t2i
1fza
2zta
1jl7
1cdc
4e3j
6gsp
1unz
1u8j
6wvw
1rjc
1la6
2g9j
1r3k
1bao
6u46
2i9e
4hw3
1ucj
4oq5
1iel
7ave
1buj
1nby
1n73
4h92
3v2n
1ikp
2nrn
1bz1
5gzp
2r4w
2ypi
2nps
5ec1
1y83
1kqi
1lup
4h90
3hoh
1i3i
1bww
2guv
1rpx
7avg
1y4f
3dvg
2yj1
2g75
3ord
4htk
1rgf
3s66
1yvs
1gpw
1ehs
1k3k
5whi
4j5g
1ehd
1u8h
1unx
1la4
1ny0
1c3b
1c08
6mh2
1s6r
3kj0
1oqm
6oqd
1fzc
2v95
1ch4
4kds
1h8n
2zm7
6vdg
4lv1
3pf3
3iih
4wb4
1h88
2zma


6mcy
1sgk
4rcx
2vdy
2vff
3e4h
3pzf
1s0h
3sh7
1l2s
4c41
1ecn
1kly
5ggw
1pz5
1ypi
5k2q
5bnk
2kf6
1gmp
1ygw
2grh
1bse
1gcl
2e0h
3fdm
1i85
3o86
1nh4
1by7
1iig
1ik9
1w5g
4mqi
1bzz
3v2q
1buu
1bu4
4jxw
1i0x
3o2d
1igl
3vhb
1ybk
1i2g
3mk8
1rgl
1r2g
1nwg
1ll9
5ku9
1b6d
1kir
1lmk
2w0f
1efe
1ulm
3th6
5y2h
1u9f
1ke0
4tim
5brb
4z0j
5fft
1s69
6oqn
4wy4
1ce9
2hjf
1xqj
2cwg
1dl0
1ar2
8tim
2dp3
1h8s
1dxt
1a2y
3ihf
1a00
2zlx
1jto
1q9b
6hbi
1x1w
4ieh
1sw7
3uwu
1vhp
3ifm
1hbw
3pia
2qsp
6str
1vib
2aw2
1tpu
2dhb
7rnt
2m4x
1qo2
1fl3
2zly
1qa9
3nmm
1q6l
1a01
4y90
3lve
3kqg
2z8a
1dxu
4blm
2x52
1hz1
1hyk
1jbt
4wuw
3bl2
4kg5
3bls
1fu1
3bm6
2a93
1cdj
6cg9
1fys
2zv6
6pqk
1nym
1u91
2bpu
4aq3
2ibl
1u9g
6nxw
1gsp
1bni
1bmr
1rgz
1nwq
2imt
6u67
1re3
6r2o
1nb1
1i2f
1wob
1ifi
4haa
4jxv
1lyx
1i3u
1iee
7az9
3v2p
1g0a
2knm
4msw
2it7
5gy6
1lu8
1pfi
4hoh
2e42
3v2r
3zr4
1g1p
1ltj
1i2s
3dvn
1piq
2k7g
3cql
2kch
2gtl
4dwt
1ifk
1efq
6u65
1r2d
1iqb
5eyw
6ip9
1low
1r2s
3zk6
1kiq
3ck4
1uo4
1uln
3dlw
1u93
43c9
1box
2n1t
1k0y
1ke3
1cdh
